In [4]:
import pandas as pd
import math

In [2]:
df = pd.read_csv('pricing_optimization.csv')

C:\Users\Shiva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
df = df[(df['trip_duration_mins']>=0) & (df['trip_duration_mins']<180)]

In [50]:
df.shape

(574407, 23)

In [52]:
#Creating 30 mins time block that walk-up users have to pay for
df['time_block_count'] = df['trip_duration_mins'].apply(lambda trip_duration_mins: math.ceil(trip_duration_mins/30))

#Creating another feature for 30 minute blocks for passholders that excludes first 30 mintues
df['time_block_count_post_free'] = df['trip_duration_mins'].apply(lambda trip_duration_mins: math.ceil(trip_duration_mins/30))-1

In [53]:
agg_pass_groups = pd.DataFrame()
#agg_pass_groups['passholder_type'] = ['annual','monthly','daily','walk-up']
agg_pass_groups = df.groupby('passholder_type').agg({
    'trip_id' : 'count',
    'walk_up' : 'sum',
    'time_block_count' : 'sum',
    'time_block_count_post_free' : 'sum',
    'trip_duration_mins' : 'sum'
}).rename(columns = {
    'trip_id' : 'total_trips',
    'walk_up' : 'pay_count_walk_up',
    'time_block_count' : 'total_30min_blocks',
    'time_block_count_post_free' : 'total_30min_blocks_post_free',
    'trip_duration_mins' : 'total_minutes'
}).reset_index()
#agg_pass_groups['passholder_type'] = ['annual','monthly','daily','walk-up']
agg_pass_groups.index = agg_pass_groups['passholder_type']
agg_pass_groups = agg_pass_groups.drop('passholder_type',1)
agg_pass_groups

,total_trips,pay_count_walk_up,total_30min_blocks,total_30min_blocks_post_free,total_minutes
passholder_type,,,,,
Flex Pass,23896,0,25802,1906,298054.0
Monthly Pass,333351,0,350693,17342,3751245.0
One Day Pass,20420,0,37563,17143,821889.0
Walk-up,196740,196740,325362,128622,6730603.0


We are however missing the number of passholders to implement revenue gained from selling passes. As we do not have unique identifiers for passholders, we refer to the data page of LA Metro bikes here: https://bikeshare.metro.net/about/data/.

With roughly 689,741 trips completed and 66,052 passes sold since it's inception in July 2016, it would be safe to expect roughly 23,000 passes sold during the year-long period of this analysis.

We will estimate the number of different passes sold from a breakeven perspective. As most trips (>60%) are "last mile" trips from metro stops and the median duration of trips is 12 minutes, we conduct the following breakeven analysis for trips under 30 minutes

In [54]:
## PREVIOUS PRICING MODEL
# Rides to break even = Pass price / (Price to ride without pass - Price to ride with pass)
# Passes sold = Total rides / Rides to break even

price_wo_pass = 3.5
price_w_pass_day = 0
price_w_pass_month = 0
price_w_pass_annual = 1.75

# Average difference between passholders and walk-up riders across previous and current pricing.
# This is represented by the difference in prices per 30 minute block since pass holders get the first 30 minutes free.
# As current data is from before the price change, previous prices are used to estimate number of passes.

day_diff = price_wo_pass - price_w_pass_day
month_diff = price_wo_pass - price_w_pass_month
annual_diff = price_wo_pass - price_w_pass_annual

# Prices of passes.

day_pass_price = 7
month_pass_price = 20
annual_pass_price = 40

# Estimated rides required for breakeven.

rides_day_pass_estimate = day_pass_price / day_diff
rides_month_pass_estimate = month_pass_price / month_diff
rides_annual_pass_estimate = annual_pass_price / annual_diff

print(" Rides per day required for daily pass:",rides_day_pass_estimate,'\n', 
      "Rides per month required for monthly pass:", rides_month_pass_estimate,'\n', 
      "Rides per year required for annual pass:", rides_annual_pass_estimate)

 Rides per day required for daily pass: 2.0 
 Rides per month required for monthly pass: 5.714285714285714 
 Rides per year required for annual pass: 22.857142857142858


Rides per day required for daily pass: 2.0 
 Rides per month required for monthly pass: 5.714285714285714 
 Rides per year required for annual pass: 22.857142857142858
Looking at the number of rides required of breakeven and assessing the numbers, the numbers seem to be on the low side.

Daily pass holders are likely to perform more than a round trip if they were purchasing a day pass.

Monthly pass holders are likely to be the occasional commuters and even at a conservative 2 rides/week to work would tally 8 rides a month.

Annual pass holders are the hardest to gauge due to the low $40 price point of the flex point from the previous pricing plan. For now, they'll be scaled to the same factor as day and monthly pass numbers.

Scaling daily passes up to a conservative 3 rides is a 50% increase in rides while scaling monthly rides up to a conservative 8 rides is a 40% increase. For conservative estimates, we scale the three estimates by a common factor of 40%.

In [55]:
rides_day_pass_estimate *= 1.4
rides_month_pass_estimate *= 1.4
rides_annual_pass_estimate *= 1.4

print("Rides per day required for daily pass (scaled):",rides_day_pass_estimate,'\n', 
      "Rides per month required for monthly pass (scaled):", rides_month_pass_estimate,'\n', 
      "Rides per year required for annual pass (scaled):", rides_annual_pass_estimate)

Rides per day required for daily pass (scaled): 2.8 
 Rides per month required for monthly pass (scaled): 8.0 
 Rides per year required for annual pass (scaled): 32.0


In [57]:
# Divide total trips by rides required for breakeven per pass duration (day/month/year) to get estimated number
# of passholders.

passes_sold_day = math.ceil(agg_pass_groups.loc['One Day Pass','total_trips']/rides_day_pass_estimate)
passes_sold_month = math.ceil(agg_pass_groups.loc['Monthly Pass','total_trips']/rides_month_pass_estimate)
passes_sold_annual = math.ceil(agg_pass_groups.loc['Flex Pass','total_trips']/rides_annual_pass_estimate)
passes_sold_total = passes_sold_day + passes_sold_month + passes_sold_annual

agg_pass_groups['passes_sold'] = [passes_sold_annual, passes_sold_month, passes_sold_day, 0]

print("Estimated passes sold for daily pass:",passes_sold_day,'\n', 
      "Estimated passes sold for monthly pass:", passes_sold_month,'\n', 
      "Estimated passes sold for annual pass:", passes_sold_annual,'\n',
     "Total passes sold:", passes_sold_total)

Estimated passes sold for daily pass: 7293 
 Estimated passes sold for monthly pass: 41669 
 Estimated passes sold for annual pass: 747 
 Total passes sold: 49709


This is roughly inline with our previous estimate of 20,000.

Lets now formulate the objective function for this optimization model to maximum total revenue from pass sales and rides.

The total revenue can be split into the following:

Total revenue from walk up users = Price for 30 minutes of usage * Total 30 minute blocks by walk up users
Total revenue from day, month pass users = Price for 30 minutes of usage * Total 30 minute blocks after free first 30 minutes
Total revenue from selling passes = Prices of respective passes * Total passes sold in respective pass types
Now we can represent this objective function in PuLP.

In [58]:
agg_pass_groups['previous_pass_prices'] = [40, 20, 7, 0]
agg_pass_groups['new_pass_prices'] = [150,17, 5, 0]
agg_pass_groups['average_charged_blocks_per_ride'] = agg_pass_groups['total_30min_blocks_post_free'] / agg_pass_groups['total_trips']
agg_pass_groups['percent_total_trips'] = agg_pass_groups['total_trips']/sum(agg_pass_groups['total_trips'])
agg_pass_groups

,total_trips,pay_count_walk_up,total_30min_blocks,total_30min_blocks_post_free,total_minutes,passes_sold,previous_pass_prices,new_pass_prices,average_charged_blocks_per_ride,percent_total_trips
passholder_type,,,,,,,,,,
Flex Pass,23896,0,25802,1906,298054.0,747,40,150,0.079762,0.041601
Monthly Pass,333351,0,350693,17342,3751245.0,41669,20,17,0.052023,0.580339
One Day Pass,20420,0,37563,17143,821889.0,7293,7,5,0.839520,0.035550
Walk-up,196740,196740,325362,128622,6730603.0,0,0,0,0.653766,0.342510


Let's first run a quick calculation to see what our current total revenue is with the estimated number of passholders over the past year.

The old pricing scheme is used here to estimate the price. Here, annual passholders pay dollar 40 a year for a flex pass and dollar 1.75 for every half hour. The first half hour is not free.

In [61]:
current_tot_rev = (sum(3.5 * agg_pass_groups['pay_count_walk_up']) 
                   + sum(1.75 * agg_pass_groups['total_30min_blocks_post_free'])
                   + 1.75 * (agg_pass_groups['total_30min_blocks'] ['Flex Pass']
                                            - agg_pass_groups['total_30min_blocks_post_free']['Flex Pass'])
                   + sum(agg_pass_groups['passes_sold'] * agg_pass_groups['previous_pass_prices']))
print('Total revenue: $', current_tot_rev)

Total revenue: $ 1933491.75


## Basic linear optimization with PuLP

Let's first try basic linear optimizations with PuLP before moving on to more advanced quadratic programming.

While the price and demand relationship is probably the most interesting one, another question that could be asked is if all plans are necessary for revenue maximization. We thus create binary variables for each plan and run a basic optimization over both previous and current price plans.

Given that riders have already been complaining about the $3.50 per half hour block rate, any dropping of passes in the past pricing scheme would have led to a precipitous drop in ridership. Furthermore, given the fierce competition in the area of bike and scooter sharing, the dropping of certain passes could simply lead to riders moving to other competitors.

We assume a baseline attrition of 30% for each category and adjust the parameters as we run the model.

In [63]:
from pulp import *

In [65]:
previous_walk_up = 3.5
previous_pass = 1.75

# Model version 1 for previous pricing

model_v1 = pulp.LpProblem("Revenue Maximization Previous", pulp.LpMaximize)

# List of all binary variables.

day_yes = pulp.LpVariable('day_yes', cat='Binary')

day_no = pulp.LpVariable('day_no', cat='Binary')

month_yes = pulp.LpVariable('month_yes', cat='Binary')

month_no = pulp.LpVariable('month_no', cat='Binary')

annual_yes = pulp.LpVariable('annual_yes', cat='Binary')

annual_no = pulp.LpVariable('annual_no', cat='Binary')

# Objective function.
# Revenue for walk ups
walk_up_rev_block = previous_walk_up * agg_pass_groups.loc['Walk-up','total_30min_blocks']

# Revenue for pass sales.
trip_rev_day_pass = agg_pass_groups.loc['One Day Pass', 'total_30min_blocks_post_free'] * day_yes * previous_pass
trip_rev_month_pass = agg_pass_groups.loc['Monthly Pass', 'total_30min_blocks_post_free'] * month_yes * previous_pass
trip_rev_annual_pass = agg_pass_groups.loc['Flex Pass', 'total_30min_blocks'] * annual_yes * previous_pass

# Revenue for pass sales is pass is cancelled.
trip_rev_day_no_pass = agg_pass_groups.loc['One Day Pass', 'total_30min_blocks'] * previous_walk_up * day_no * 0.7
trip_rev_month_no_pass = agg_pass_groups.loc['Monthly Pass', 'total_30min_blocks'] * previous_walk_up * month_no * 0.7
trip_rev_annual_no_pass = agg_pass_groups.loc['Flex Pass', 'total_30min_blocks'] * previous_walk_up * annual_no * 0.7

# Revenue for sales of passes.
pass_sales_rev_day = (day_yes * agg_pass_groups.loc['One Day Pass','previous_pass_prices'] 
                      * agg_pass_groups.loc['One Day Pass','passes_sold'])
pass_sales_rev_month = (month_yes * agg_pass_groups.loc['Monthly Pass','previous_pass_prices'] 
                        * agg_pass_groups.loc['Monthly Pass','passes_sold'])
pass_sales_rev_annual = (annual_yes * agg_pass_groups.loc['Flex Pass','previous_pass_prices'] 
                         * agg_pass_groups.loc['Flex Pass','passes_sold'])

# Adding objective function to model
model_v1 += pulp.lpSum(walk_up_rev_block + trip_rev_day_pass + trip_rev_day_no_pass + trip_rev_month_pass 
                       + trip_rev_month_no_pass + trip_rev_annual_pass + trip_rev_annual_no_pass 
                       + pass_sales_rev_day + pass_sales_rev_month + pass_sales_rev_annual)

# Constraints such that only one of each binary pair can be true.
model_v1 += day_yes + day_no == 1
model_v1 += month_yes + month_no == 1
model_v1 += annual_yes + annual_no == 1

optimization_result = model_v1.solve()

assert optimization_result == pulp.LpStatusOptimal
print("Status:", LpStatus[model_v1.status])
print("Optimal Solution to the problem: ", value(model_v1.objective))
print ("Individual decision_variables: ")
for v in model_v1.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
Optimal Solution to the problem:  2169558.35
Individual decision_variables: 
annual_no = 0.0
annual_yes = 1.0
day_no = 1.0
day_yes = 0.0
month_no = 0.0
month_yes = 1.0


In [66]:
print(model_v1)

Revenue Maximization Previous:
MAXIMIZE
63214.899999999994*annual_no + 75033.5*annual_yes + 92029.34999999999*day_no + 81051.25*day_yes + 859197.85*month_no + 863728.5*month_yes + 1138767.0
SUBJECT TO
_C1: day_no + day_yes = 1

_C2: month_no + month_yes = 1

_C3: annual_no + annual_yes = 1

VARIABLES
0 <= annual_no <= 1 Integer
0 <= annual_yes <= 1 Integer
0 <= day_no <= 1 Integer
0 <= day_yes <= 1 Integer
0 <= month_no <= 1 Integer
0 <= month_yes <= 1 Integer

